We have learned how to implement a model and train it from stratch. However, for pratical machine learning tasks, we often reuse popular model architectures and load pre-trained weights on famous large datasets and fine-tune with our own small, specific dataset, which is called transfer learning. In this notebook we will load a pre-trained ConvNet and use it for our own goal.

In [ ]:
import tensorflow.keras as keras
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import utils
import os

You can find famous ConvNets in keras.applications.

In [ ]:
baseModel = keras.applications.VGG16(input_shape=(224,224,3), weights='imagenet',include_top=False)

In [ ]:
baseModel.trainable = False
cnnModel = keras.Sequential([baseModel,layers.Flatten(),
                  layers.Dense(units = 512, activation = 'relu'), 
                  layers.Dense(units = 3, activation = 'softmax')])
cnnModel.summary()
print("Number of layers in VGG16: ", len(baseModel.layers))

In [ ]:
filenames = []
labels = []
class_names = ['bird','cat','dog']
for filename in os.listdir("./data"):
    if filename.startswith("bird"):
        labels.append(0)
        filenames.append(os.path.join("./data",filename))
    elif filename.startswith("cat"):
        labels.append(1)
        filenames.append(os.path.join("./data",filename))
    elif filename.startswith("dog"):
        labels.append(2)
        filenames.append(os.path.join("./data",filename))
print(filenames)
print(labels)
path_dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))

In [ ]:
def load_and_preprocess_image(filename, label):
    raw_image = tf.io.read_file(filename)
    image_tensor = tf.image.decode_jpeg(raw_image)
    image_tensor = tf.image.resize(image_tensor, [224, 224])
    image_tensor /= 255.0
    return image_tensor,label

In [ ]:
imageDataset = path_dataset.map(load_and_preprocess_image).shuffle(buffer_size=100)
trainDataset = imageDataset.take(int(len(filenames)* 0.7))
testDataset = imageDataset.skip(int(len(filenames)* 0.7))

In [ ]:
cnnModel.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.0001),
             loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
    tf.keras.callbacks.ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
]
history = cnnModel.fit(trainDataset.shuffle(5).batch(5), epochs=20, validation_data=testDataset.repeat().batch(10), validation_steps=10, callbacks = callbacks)

In [ ]:
# history record the accuracy/loss on training set/validation set after every epoch 
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['training', 'valivation'], loc='upper left')
plt.show()

In [ ]:
result = cnnModel.evaluate(testDataset.batch(10))

In [ ]:
predict_result = tf.argmax(cnnModel.predict(testDataset.take(9).batch(9)),axis=1)

In [ ]:
predict_result.numpy()

In [ ]:
utils.show_images(testDataset, class_names, predict_result)